# This notebook tracks the convergence of the multipoles of the protein
First load all the files.

In [ ]:
from glob import glob
from os.path import basename
background = "QMMMData/With Background-"
nobackground = "QMMMData/No Background-"

systems = [basename(x) for x in glob(background+"*")]
systems = [x.replace("With Background-", "").replace(".csv","") for x in systems]
print(systems)

In [ ]:
from pandas import read_csv

data_set = {}
data_set["Background"] = {}
data_set["No Background"] = {}

for back_type, fpath in zip(data_set, [background, nobackground]):
    data_set[back_type] = {}
    for s in systems:
        fname = fpath+s+".csv"
        data_set[back_type][s] = read_csv(fpath+s+".csv")

In [ ]:
from pandas import DataFrame
from numpy import array
from numpy.linalg import norm
headers = ["Threshold", "Q0", "Q1", "Q2"]
error_set = {}
for back_type in data_set:
    error_set[back_type] = {}
    for s in systems:
        error_set[back_type][s] = DataFrame(columns=headers)
        
        row = data_set[back_type][s].loc[0]
        Q0_ref = array(row[1])
        Q1_ref = array(row[2:5])
        Q2_ref = array(row[5:])
        for index, row in data_set[back_type][s].iterrows():
            if row[0] == "Reference":
                continue
            Q0 = array(row[1])
            Q1 = array(row[2:5])
            Q2 = array(row[5:])
            error_row = [10**(-1*int(row[0])), norm(Q0 - Q0_ref)/norm(Q0_ref), 
                        norm(Q1 - Q1_ref)/norm(Q1_ref), norm(Q2 - Q2_ref)/norm(Q2_ref)]
            new_row = DataFrame([error_row], columns=headers)
            error_set[back_type][s] = error_set[back_type][s].append(new_row)

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(2,3, figsize=(12,6))

fs=14

markers = ['^-', 'h-', 'o-', 'x-', '|-']

for i, back_type in enumerate(data_set):
    axs[i,0].set_ylabel(back_type, fontsize=fs)
    for j in range(0, 3):
        axs[i,j].set_xscale("log")
        axs[i,j].set_yscale("log")
        axs[i,j].set_ylim([1e-6, 1])
for j, order in enumerate(["Q0", "Q1", "Q2"]):
    axs[-1,j].set_xlabel(order,fontsize=fs)

for i, back_type in enumerate(data_set):
    for s, m in zip(systems, markers):
        thresh = list(error_set[back_type][s]["Threshold"])
        for j, col in enumerate(["Q0", "Q1", "Q2"]):
            val = list(error_set[back_type][s][col])
            axs[i,j].plot(thresh, val, m, label=s)
            axs[i,j].set_xlim(max(thresh),min(thresh))

# fig.tight_layout()
axs[0,0].legend(loc="upper center", bbox_to_anchor=(1.75,1.25),ncol=len(systems))
fig.savefig("Pictures/QMMMConvergence.eps")